# ULKB Initialization

In [2]:
from ulkb import *                   # import ULKB Logic namespace
settings.serializer.ulkb.ensure_ascii = True

# FOL reasoning

In [27]:
a             = TypeVariable('a') # a : *
ab            = FunctionType(a, BoolType()) # a -> bool : *
aab           = FunctionType(a, a, BoolType()) # a -> a -> bool : *

lives         = new_constant("lives", ab)
killed        = new_constant("killed", aab)
hates         = new_constant("hates", aab)
richer        = new_constant("richer", aab)

agatha        = new_constant('agatha', a)
butler        = new_constant('butler', a)
charles       = new_constant('charles', a)

x, y = Variables('x', 'y', a)

ax1  = Exists(x, And(lives(x), killed(x,agatha)))
ax3  = Forall(x, Implies(lives(x),Or(equal(x,agatha),equal(x,butler),equal(x,charles))))
ax4  = Forall(x, Forall(y, Implies(killed(x,y),hates(x,y))))
ax5  = Forall(x, y, Implies(killed(x,y), Not(richer(x,y))))
ax6  = Forall(x, Implies(hates(agatha,x), Not(hates(charles,x))))
ax7  = Forall(x, Implies(Not(equal(x,butler)), hates(agatha,x)))
ax8  = Forall(x, Implies(Not(richer(x, agatha)), hates(butler,x)))
ax9  = Forall(x, Implies(hates(agatha,x), hates(butler,x)))
ax10 = Not(Exists(x, Forall(y, Implies(lives(y),hates(x,y)))))
ax11 = Not(equal(agatha,butler))

for ax in [ax1,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11]: 
    new_axiom(ax)

show_axioms()

26	axiom exists x, lives x and killed x agatha
27	axiom forall x, lives x -> x = agatha or x = butler or x = charles
28	axiom forall x y, killed x y -> hates x y
29	axiom forall x y, killed x y -> not richer x y
30	axiom forall x, hates agatha x -> not hates charles x
31	axiom forall x, x != butler -> hates agatha x
32	axiom forall x, not richer x agatha -> hates butler x
33	axiom forall x, hates agatha x -> hates butler x
34	axiom not (exists x, forall y, lives y -> hates x y)
35	axiom agatha != butler


In [28]:
print(RuleZ3(killed(agatha, agatha)))

|- killed agatha agatha


# MRS Logic

In [35]:
! killall ukb_wsd
%set_env ACE_GRAMMAR erg.dat
import mrs_logic
    
ctx = mrs_logic.get_current_context()
ctx.options['to_ulkb']['universe_type'] = a   # set the type of the (FOL) universe to "a"
ctx.options['to_ulkb']['drop_uvars'] = True   # delete "u" vars

No matching processes belonging to you were found
env: ACE_GRAMMAR=erg.dat


In [36]:
sol = next(mrs_logic.solve('Someone who lives in Mansion killed Agatha.'))
form = sol.to_ulkb()
print(sol.mrs)
print(form)
form.constants

[ TOP: h0
  INDEX: e2
  RELS: < [ person LBL: h4 ARG0: x3 ]
          [ _some_q LBL: h5 ARG0: x3 RSTR: h6 BODY: h7 ]
          [ _live_v_1 LBL: h4 ARG0: e8 ARG1: x3 ]
          [ _in_p_dir LBL: h4 ARG0: e9 ARG1: e8 ARG2: x10 ]
          [ proper_q LBL: h11 ARG0: x10 RSTR: h12 BODY: h13 ]
          [ named LBL: h14 ARG0: x10 CARG: "Mansion" ]
          [ _kill_v_1 LBL: h1 ARG0: e2 ARG1: x3 ARG2: x16 ]
          [ proper_q LBL: h17 ARG0: x16 RSTR: h18 BODY: h19 ]
          [ named LBL: h20 ARG0: x16 CARG: "Agatha" ] >
  HCONS: < h0 qeq h1 h6 qeq h4 h12 qeq h14 h18 qeq h20 > ]
(exists x10, Mansion = x10 and (exists x3, (exists e8 e9, person x3 and _live_v_1 e8 x3 and _in_p_dir e9 e8 x10) and (exists x16, Agatha = x16 and (exists e2, _kill_v_1 e2 x3 x16)))) : bool


frozenset({Agatha : a,
           Mansion : a,
           _in_p_dir : a -> a -> a -> bool,
           _kill_v_1 : a -> a -> a -> bool,
           _live_v_1 : a -> a -> bool,
           and : bool -> bool -> bool,
           equal : a -> a -> bool,
           exists : (a -> bool) -> bool,
           person : a -> bool})

In [37]:
x, y, e1, e2 = Variables('x', 'y', 'e1', 'e2', a)

Agatha    = Constant('Agatha', a)
Mansion   = Constant('Mansion', a)
person    = Constant('person', FunctionType(a, bool))
_kill_v_1 = Constant('_kill_v_1', FunctionType(a, a, a, bool))
_in_p_dir = Constant('_in_p_dir', FunctionType(a, a, a, bool))
_live_v_1 = Constant('_live_v_1', FunctionType(a, a, bool))

new_axiom(equal(Agatha, agatha))
new_axiom(Forall(x, Iff(Exists(e1, e2, y, And(person(x), _live_v_1(e1, x), _in_p_dir(e2, e1, y),Equal(Mansion,y))), 
                        lives(x))))

new_axiom(Forall(x,y,Iff(Exists(e1, _kill_v_1(e1, x, y)), killed(x,y))))

show_axioms()

26	axiom exists x, lives x and killed x agatha
27	axiom forall x, lives x -> x = agatha or x = butler or x = charles
28	axiom forall x y, killed x y -> hates x y
29	axiom forall x y, killed x y -> not richer x y
30	axiom forall x, hates agatha x -> not hates charles x
31	axiom forall x, x != butler -> hates agatha x
32	axiom forall x, not richer x agatha -> hates butler x
33	axiom forall x, hates agatha x -> hates butler x
34	axiom not (exists x, forall y, lives y -> hates x y)
35	axiom agatha != butler
36	axiom Agatha = agatha
37	axiom forall x, (exists e1 e2 y, person x and _live_v_1 e1 x and _in_p_dir e2 e1 y and Mansion = y) <-> lives x
38	axiom forall x y, (exists e1, _kill_v_1 e1 x y) <-> killed x y


In [39]:
RuleZ3(form)

|- exists x10, Mansion = x10 and (exists x3, (exists e8 e9, person x3 and _live_v_1 e8 x3 and _in_p_dir e9 e8 x10) and (exists x16, Agatha = x16 and (exists e2, _kill_v_1 e2 x3 x16)))